In [1]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt
import io as io
#from adjustText import adjust_text
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

In [2]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
iquedata

,Experiment,Analysis,Plate,Export Date and Time,Well ID,Row,Column,Well Type,Count of All Wells,Median BL1-H of singlets,Median RL1-H of singlets,Count of singlets
0,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,A01,A,1.0,Positive,2770.0,14894618.0,1546077.0,472.0
1,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,B01,B,1.0,Positive,4560.0,9850253.0,1594571.0,1113.0
2,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,C01,C,1.0,Positive,4179.0,11827091.0,2029051.0,872.0
3,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,D01,D,1.0,Positive,2930.0,13698218.0,2174324.0,485.0
4,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,E01,E,1.0,Positive,5715.0,10738373.0,638947.0,1719.0
...,...,...,...,...,...,...,...,...,...,...,...,...
220,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,M19,M,19.0,Negative,1735.0,14351.5,260783.5,838.0
221,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,N19,N,19.0,Negative,1921.0,14615.0,254192.0,959.0
222,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,O19,O,19.0,Negative,865.0,14057.0,176848.0,222.0
223,22_06_30 4-1BB TB693 Primary Screen + A2a Exre...,Analysis 1,4-1BB TB693 Primary,6/30/22 11:20,P19,P,19.0,Negative,2197.0,13870.5,232991.0,1094.0


In [3]:
data_cleaned = iquedata[["Plate", "Well ID", "Well Type", "Median RL1-H of singlets"]]
data_cleaned

,Plate,Well ID,Well Type,Median RL1-H of singlets
0,4-1BB TB693 Primary,A01,Positive,1546077.0
1,4-1BB TB693 Primary,B01,Positive,1594571.0
2,4-1BB TB693 Primary,C01,Positive,2029051.0
3,4-1BB TB693 Primary,D01,Positive,2174324.0
4,4-1BB TB693 Primary,E01,Positive,638947.0
...,...,...,...,...
220,4-1BB TB693 Primary,M19,Negative,260783.5
221,4-1BB TB693 Primary,N19,Negative,254192.0
222,4-1BB TB693 Primary,O19,Negative,176848.0
223,4-1BB TB693 Primary,P19,Negative,232991.0


In [4]:
#Return data frame with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating the well type of interest (marked in iQue3 software, or manually changed
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_MFI(well_type_string):
    selected_wells = clean_for_well_type(well_type_string)
    MFIs = selected_wells["Median RL1-H of singlets"]
    firsts = MFIs.iloc[::2]
    seconds = MFIs.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    average = firsts + seconds
    average = average/2
    return average

#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
def calculate_MFI_Ratio(positive, negative):
    MFI_ratio = positive/negative
    original_positive = clean_for_well_type("Positive")
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive["MFI Ratio"] = MFI_ratio
    MFIs_Antibody = pd.concat([original_positive, ab_sheet], axis = 1)
    MFIs_Antibody.index += 1
    return MFIs_Antibody[["Plate", "Well ID", "Well Type", "MFI Ratio", "DNA"]] 

In [5]:
averagePositive = average_MFI("Positive")
averageNegative = average_MFI("Negative")

In [6]:
#Create new DF for MFI ratios(don't modify originals)
MFI_Ratios_Antibody = calculate_MFI_Ratio(averagePositive, averageNegative)
MFI_Ratios_Antibody
MFI_Ratios_Antibody.to_excel(elnname, sheet_name='Analyzed Data')
AllLeadsSorted = MFI_Ratios_Antibody.sort_values(by="MFI Ratio", ascending = False)
AllLeadsSorted

,Plate,Well ID,Well Type,MFI Ratio,DNA
56,4-1BB TB693 Primary,O07,Positive,55.382958,Goat PAB
7,4-1BB TB693 Primary,M01,Positive,42.013631,TB693-012
5,4-1BB TB693 Primary,I01,Positive,41.658302,TB693-009
1,4-1BB TB693 Primary,A01,Positive,35.733239,TB693-001
4,4-1BB TB693 Primary,G01,Positive,32.053994,TB693-008
8,4-1BB TB693 Primary,O01,Positive,30.282656,TB693-013
31,4-1BB TB693 Primary,M04,Positive,26.933239,TB693-052
33,4-1BB TB693 Primary,A05,Positive,26.916019,TB693-054
28,4-1BB TB693 Primary,G04,Positive,25.596823,TB693-047
32,4-1BB TB693 Primary,O04,Positive,24.571091,TB693-053


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))